# Bem vindo ao EEG Stats!

**LEIA ATENTAMENTE ANTES DE COMEÇAR**


Certifique-se de seguir os seguintes passos:

1.   Você está rodando este arquivo (EEG_Stats.ipynb) no Google Colaboratory (colab.research.google.com).
2.   Crie uma pasta chamada "your_folders_here" em seu Google Drive.
3.   Dentro desta pasta que criou copie suas pastas contendo os arquivos Txts organizados conforme explicação abaixo.
4. Siga os procedimentos exatamente como indicado nos textos abaixo e só altere as células de código quando for pedido.




**1 - Como organizar os arquivos TXTs:**

Se você está utilizando este script, provavelmente você possui um modelo experimental com duas variáveis independentes.
A nomeclatura de suas pastas deve seguir a seguinte regra:
1. A pasta de ter somente dois numeros em seu nome
2. O primeiro número representa o número do subnível da primera variável independente
3. O segundo número representa o número do subnível da segunda variável independente

Desta forma, se você tiver 3 subníveis diferentes na primeira variável e 2 na segunda, você deverá ter as seguintes pastas: 11, 12, 21, 22, 31, 32.


**Exemplo**

Digamos que seu desenho experimental consista em 3 tarefas diferentes, e dois momentos (pré e pós). 

A pasta "11" deverá conter seus arquivos txt referente a tarefa 1 momento pré;

A pasta "12" deverá conter seus arquivos txt referente a tarefa 1 momento pós;

A pasta "21" deverá conter seus arquivos txt referente a tarefa 2 momento pré;

A pasta "22" deverá conter seus arquivos txt referente a tarefa 2 momento pós;

A pasta "31" deverá conter seus arquivos txt referente a tarefa 3 momento pré;

A pasta "32" deverá conter seus arquivos txt referente a tarefa 3 momento pós;

Assumindo que esta estrutura de pastas e arquivos estão em seu Google Drive vamos seguir para o próximo passo.


**2 - Instalação do Pandas Profiling**

Uma das bibliotecas que vamos utilizar não é padrão do Google Colab e precisamos fazer a instalação.

Para isso, Execute a célula abaixo. 

OBS: Para executar a célula, selecione e clique no botão play no canto superior esquerdo da mesma, ou simplesmente a selecione e pressione CTRL+ENTER.

Ao final aparecerá um botão com a seguinte mensagem "RESTART RUNTIME".

Clique no botão e siga para a próxima etapa.
Caso o botão não apareça siga para a próxima etapa.

In [0]:
# instalando o pandas profiling
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

**3 - Montagem do Google Drive**

Execute a célula abaixo. Para isso selecione a célula e clique no botão play no canto superior esquerdo da mesma, ou simplesmente a selecione e pressione CTRL+ENTER.

Feito isto, irá aperecer uma mensagem com um link e um campo para digitar o código de autorização.
Clique no link e siga os passos do Google para autorizar o acesso ao seu Google Drive (não se preocupe, o acesso se encerra após a finalização do script e só é utilizado para acessar os arquivos Txt e para salvar os resultados).

Ao final do processo o Google disponibilizará uma série de caracteres. Clique no icone para copiar e os cole no campo do código de autorização da célula abaixo que você acabou de executar e pressione ENTER.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**4 - Defina as variáveis do desenho experimental utilizado**

Ajuste as variáveis de acordo com o seu desenho experimental.

Abaixo segue a célula preenchida para o desenho experimental citado a cima.

In [0]:
# Design Infos
bonferroni = 7 #Bonferroni correction factor
frequency = 'Beta' # Name of the analysed frequency
indVar1_SubLevels = ['Execution', 'Action Observation', 'Motor Imagery']
indVar2_SubLevels = ['Pre','Post'] # Indicates the number of sub-levels of the independent variable 1

lista_eletrodos = ['Fp1','Fp2','F7','F3','Fz','F4','F8','T3','C3',
            'Cz','C4','T4','T5','P3','Pz','P4','T6','O1','Oz','O2']

**5 - Defina o caminho (*path*) para a pasta de trabalho no Google Drive**

Esta etapa é muito importante!
Clique na pasta na coluna a esquerda do colab para abrir a estrutra do seu google drive.
Navegue até a sua pasta "your_folders_here".
Selecione a pasta que contém a pasta "your_folders_here", clique nos 3 pontos a direita da mesma e selecione a opção "Copy path".

Agora altere a célula abaixo substituindo o texto *SUBSTITUA ESSE TEXTO* selecionando o mesmo e pressionando CRTL+V.

Sua célula deve conter somente a palavra *path* seguida do sinal de igual e com o valor colado entre aspas.

Ex: path = "/content/drive/My Drive/IPUB/EEG_Stats"

In [0]:
path = "SUBSTITUA ESSE TEXTO"

Feito isso, vá no menu superior na opção "Runtime" e selecione a opção "Restart and run all".

Aguarde a execução do script. quando o mesmo encerrar uma mensagem dizendo "ENDED SUCCESSFULLY" irá aparece no final desta página.

# NÃO ALTERAR DESTE PONTO EM DIANTE!

**Importação das demais Bibliotecas**

Nesta etapa vamos importar as demais bibliotecas.
Basta executar a célula abaixo.

In [0]:
import os
import csv
import numpy as np
import pandas as pd
import scipy
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from statsmodels.formula.api import ols
from tabulate import tabulate
from pandas_profiling import ProfileReport

Verifica se os diretórios para salvar os resultados existem e os cria se necessário

In [0]:
'''Checks if results dir exists and creates it if necessary'''
if not os.path.exists(path + '/results'):
    os.makedirs(path + '/results')
if not os.path.exists(path + '/results/plots'):
    os.makedirs(path + '/results/plots')
if not os.path.exists(path + '/results/boxplots/'):
    os.makedirs(path + '/results/boxplots/')

Cria uma lista baseada nos diretórios dos arquivos TXTs

In [0]:
'''Creates a list of the usefull directories to concatenate'''
listdir = ([x[0] for x in os.walk(path + '/your_folders_here/')]) #create list with all sub-directories
listdir.remove(path + '/your_folders_here/') #remove sub-directory lib
listdir = [w.replace( (path + '/your_folders_here/'), '') for w in listdir]

Cria o arquivo da planilha geral

In [0]:
'''Creates the final sheet'''
outputfile = open((path + '/results/final_sheet.csv'), 'w')
writer = csv.writer(outputfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_ALL)
#Adds the header
writer.writerow(['IndVar1','IndVar2','Fp1','Fp2','F7','F3','Fz','F4','F8','T3','C3',
            'Cz','C4','T4','T5','P3','Pz','P4','T6','O1','Oz','O2'])

Concatena os txts na planilha csv criada

In [0]:
#Concatenates each row of each txt file to the final_sheet.csv
for i in listdir:
    source = path + '/your_folders_here/' + i + '/'
    ind_var = i[0]
    dep_var = i[1]
    for txt_file in os.listdir(source):
        if txt_file.endswith('txt') == True:
            files_opened = open((source + txt_file))
            reader = csv.reader(files_opened, delimiter=' ')
            next(reader, None)  # skip the headers
            for row in reader:
                row = [n.replace('+', '') for n in row]
                writer.writerow([ind_var, dep_var] + row)
            files_opened.close() #closes txt files
outputfile.close() #closes the csv file
#prints end feedback
print("Final sheet done!")

Define a planilha criada como base de dados para a execução de todos os calculos.

In [0]:
data = pd.read_csv(path + '/results/final_sheet.csv', index_col=False) # Import final_sheet

Define as variaveis que serão utilizadas baseadas no desenho experimental

In [0]:
''' CREATE VARIABLES BASED ON DESIGN INFORMATIONS '''

indVar1 = data['IndVar1'] # Indenpendent Variable 1
indVar2 = data['IndVar2'] # Indenpendent Variable 2

# Indenpendent Variable 1 Sublevels
subLvs_id1 = {}
i=0
for x in indVar1_SubLevels:
  i += 1
  exec("iv1_SubLv%d = (data.loc[data['IndVar1'] == %d])"%(i,i))
  subLvs_id1[x] = ("iv1_SubLv%d"%i)

# Indenpendent Variable 2 Sublevels
subLvs_id2 = {}
i=0
for y in indVar2_SubLevels:
  i += 1
  exec("iv2_SubLv%d = (data.loc[data['IndVar2'] == %d])"%(i,i))
  subLvs_id2[y] = ("iv2_SubLv%d"%i)
  

# Independent Variable 1 vs 2 Sublevels
subLvs_id1id2 = {}
i = 0
for a in subLvs_id1:
  i += 1
  j = 0
  for b in subLvs_id2:
    j += 1
    exec("iv1_SubLv%d_iv2_SubLv%d = (data.loc[data['IndVar1'] == %d]).loc[data['IndVar2'] == %d]"%(i,j,i,j))
    subLvs_id1id2[(a+b)] = ("iv1_SubLv%d_iv2_SubLv%d"%(i,j))


# Spliting Dicts in Keys and lists
# indVar1 vs indVar2
keys = []
for value in list(subLvs_id1id2.values()):
  keys.append(eval(value))

lsSubLvs = []
for value in list(subLvs_id1id2.keys()):
  lsSubLvs.append(value)

# indVar1
keys1 = []
for value in list(subLvs_id1.values()):
  keys1.append(eval(value))

lsSubLvs1 = []
for value in list(subLvs_id1.keys()):
  lsSubLvs1.append(value)

# indVar2
keys2 = []
for value in list(subLvs_id2.values()):
  keys2.append(eval(value))

lsSubLvs2 = []
for value in list(subLvs_id2.keys()):
  lsSubLvs2.append(value)

# Split Keys in parts acording do IndVar2 Sublevels
iv1iv2Lvs = {} 
for i in range(len(lsSubLvs2)):
  parts = len(lsSubLvs2)
  exec('iv1iv2Lv%i=[]'%i)
  exec('iv1iv2Lv%i.append(keys[%i::parts])'%(i,i))
  iv1iv2Lvs['subLv%i'%i] = ('iv1iv2Lv%i'%i)

# Plot Styling
color_list = ['b','r','g','y','c']
icon_list = ['ks', 'ko', 'k^','kv', 'k8', 'kh', 'kd', 'kp', 'kH']
fill_list = ['none', 'full']

**Inicio da declaração das funções**

Define a função que remove outliers com Robust Sigma

In [0]:
def remove_outliers(data): # Removes outliers
    '''Creates the final sheet'''
    outputfile = open((path + '/results/final_sheet_wo_outliers.csv'), 'w')
    writer = csv.writer(outputfile, delimiter=',',quotechar='"',
                                            quoting=csv.QUOTE_ALL)
    #Adds the header
    writer.writerow(['IndVar1','IndVar2','Fp1','Fp2','F7','F3','Fz','F4','F8','T3','C3',
                'Cz','C4','T4','T5','P3','Pz','P4','T6','O1','Oz','O2'])

    def filter (fase):
        for x in lista_eletrodos:
            eletrodo = x
            i = fase[eletrodo]
            # Measure the percentile intervals and then estimate Standard Deviation of the distribution
            p90 = np.percentile(i, 90)
            p10 = np.percentile(i, 10)
            p50 = np.median(i)
            # Robust Sigma
            rSig = (p90-p50)/1.2815
            rSig = (p90-p10)/(2*1.2815)
            # Remove outliers
            filtered = i[abs(i - p50) < rSig*5]
            fase[eletrodo] = filtered
            #print (i.mean())
            i.fillna(i.mean(), inplace=True)
        return fase

    ind_var = 0
    dep_var = 0

    for fase in keys:
        filter(fase)
        np.savetxt(outputfile, fase, delimiter=',', fmt='"%1.8f"')
    outputfile.close() #closes the csv file
    print ('Outliers removed!')

Define a função que executa uma two-way ANOVA e Teste-T quando necessário

In [0]:
def run_stats_twa_tt(data): # Runs Statistics Two-way Anova and T-test when needed

    # Variables
    eletrodo = ''
    plotly_form = ' ~ C(indVar1) + C(indVar2) + C(indVar1):C(indVar2)'
    sig_con = ''
    sig_mom = ''
    sig_conxmom = ''
    p = 0.05/bonferroni #Bonferroni Correction for 7 Hypothesis

    ''' Loop for Two-way ANOVA for each Electrode '''
    for x in lista_eletrodos:
        eletrodo = x
        # Begining of ANOVA
        formula = (eletrodo + plotly_form)
        model = ols(formula, data, missing='drop').fit()
        aov_table = statsmodels.stats.anova.anova_lm(model, typ=2)
        sig_con = aov_table['PR(>F)']['C(indVar1)']
        sig_mom = aov_table['PR(>F)']['C(indVar2)']
        sig_conxmom = aov_table['PR(>F)']['C(indVar1):C(indVar2)']
        outputfile.write('\n\n\n\n' + str(model.summary()))
        outputfile.write('\n\nTwo-way Anova of Electrode: %s \n' % eletrodo)
        outputfile.write(tabulate(aov_table, headers='keys'))
        outputfile.write('\n\n')

        if (sig_conxmom < p):

            ''' T-test and Post Hoc Scheffe (POST HOC ON TESTS)'''
            '''MODIFY ACCORDING TO DESIGN INFORMATIONS'''
            idx=0
            for sublevel in indVar1_SubLevels:
              twosample_results = scipy.stats.ttest_ind(keys[idx][eletrodo], keys[idx+1][eletrodo])
              txt1 = "The t-statistic for %s " % (indVar1_SubLevels[idx])
              txt2 = "is %.3f and the p-value is %.3f.\n" % twosample_results
              outputfile.write ( txt1 + txt2 )
              idx +=1

        elif (sig_conxmom >= p and sig_con < p and sig_mom < p):
            outputfile.write ('Main effect for indVar1 and indVar2.\n')
        elif (sig_conxmom >= p and sig_mom < p):
            outputfile.write ('Main effect for indVar2.\n')
        elif (sig_conxmom >= p and sig_con < p):
            outputfile.write ('Main effect for indVar1.\n')
        else:
            outputfile.write ('No interection and no main effects.\n')
        outputfile.write('\n\n')

        # Means and STD
        i=0
        for sublevel in keys:
            x = sublevel[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            outputfile.write('Electrode:%s ' % eletrodo + lsSubLvs[i])
            outputfile.write('\n')
            outputfile.write('Mean: %s\n' % means)
            outputfile.write('Standard Deviation: %s\n' % std)
            outputfile.write('\n')
            i += 1

    outputfile.write('\n\n')

    #prints end feedback
    print("Stats done!")

Define a função que executa uma Two-way ANOVA e uma One-way ANOVA quando necessário

In [0]:
def run_stats_twa_owa(data): # Runs Statistics Two-way Anova and One-way Anova when needed
    # Variables
    eletrodo = ''
    plotly_form = ' ~ C(indVar1) + C(indVar2) + C(indVar1):C(indVar2)'
    sig_con = ''
    sig_mom = ''
    sig_conxmom = ''
    p = 0.05/bonferroni #Bonferroni correction for 7 Hypothesis

    ''' Loop for Two-way ANOVA for each Electrode '''
    for x in lista_eletrodos:
        eletrodo = x
        # Begining of Two-way ANOVA
        formula = (eletrodo + plotly_form)
        model = ols(formula, data, missing='drop').fit()
        aov_table = statsmodels.stats.anova.anova_lm(model, typ=2)
        sig_con = aov_table['PR(>F)']['C(indVar1)']
        sig_mom = aov_table['PR(>F)']['C(indVar2)']
        sig_conxmom = aov_table['PR(>F)']['C(indVar1):C(indVar2)']
        outputfile.write('\n\n\n\n' + str(model.summary()))
        outputfile.write('\n\nTwo-way Anova of Electrode: %s \n' % eletrodo)
        outputfile.write(tabulate(aov_table, headers='keys'))
        outputfile.write('\n\n')

        if (sig_conxmom < p):

            '''One-way ANOVA'''
            '''MODIFY ACCORDING TO DESIGN INFORMATIONS'''
            outputfile.write('One-way Anova:')

            idx = 0
            x = []
            for a in subLvs_id2:
              count = 0
              for b in range(len(subLvs_id1)):
                x.append(keys[count+idx][eletrodo])
                count +=1
              Fa, pa = scipy.stats.f_oneway(*x)
              txt1 = '\n%s'%indVar2_SubLevels[idx]
              txt2 = ': F statistic = {:5.3f} and probability p = {:5.3f}'.format(Fa, pa)
              outputfile.write ( txt1 + txt2 )
              x = []
              idx +=1      

        elif (sig_conxmom >= p and sig_con < p and sig_mom < p):
            outputfile.write ('Main effect for indVar1 and indVar2.\n')
        elif (sig_conxmom >= p and sig_mom < p):
            outputfile.write ('Main effect for indVar2.\n')
        elif (sig_conxmom >= p and sig_con < p):
            outputfile.write ('Main effect for indVar1.\n')
        else:
            outputfile.write ('No interection and no main effects.\n')
        outputfile.write('\n\n')

        # Means and STD
        i=0
        for sublevel in keys:
            x = sublevel[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            outputfile.write('Electrode:%s ' % eletrodo + lsSubLvs[i])
            outputfile.write('\n')
            outputfile.write('Mean: %s\n' % means)
            outputfile.write('Standard Deviation: %s\n' % std)
            outputfile.write('\n')
            i += 1

    outputfile.write('\n\n')

    #prints end feedback
    print("Stats done!")

Define a função que gera os Plots e os salva em Vetor (.eps) e Raster (.png)

In [0]:
def plot_to_files (data): # Generates Plots Vector and Raster for each electrode
    # Export path
    plot_directory = path + '/results/plots/'
    # Calc y axe limits
    ylim_min = 0
    ylim_max = 0
    for eletrodo in lista_eletrodos:
        for sublevel in keys:
            x = sublevel[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            if ((means-std) < ylim_min):
                ylim_min = (means-std)
            elif ((means+std) > ylim_max):
                ylim_max = (means+std)

    for eletrodo in lista_eletrodos:
        for sublevel in keys2:
            x = sublevel[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            if ((means-std) < ylim_min):
                ylim_min = (means-std)
            elif ((means+std) > ylim_max):
                ylim_max = (means+std)

    for eletrodo in lista_eletrodos:
        for sublevel in keys1:
            x = sublevel[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            if ((means-std) < ylim_min):
                ylim_min = (means-std)
            elif ((means+std) > ylim_max):
                ylim_max = (means+std)

    # Interaction Plots
    for eletrodo in lista_eletrodos:
        count = 0
        x_value = 0.5
        inc = True
        for sublevel in keys: # group results in pairs
            if (inc == False):
                x_value += 0.5
                inc = True
            else:
                x_value += 1
                inc = False
            count += 1
            fmt_stl = []
            ls_stl = []
            for i in range(len(indVar1_SubLevels)):
              for j in range(len(indVar2_SubLevels)):
                fmt_stl.append(icon_list[i])
                ls_stl.append(fill_list[j])
            x = sublevel[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            plt.errorbar(x_value, means, std, fmt=fmt_stl[count-1], fillstyle=ls_stl[count-1], lw=1, capsize=4, ecolor='k', markeredgewidth=1, mec='k')

        # Axes and Labels
        plt.box(on=False)
        plt.xlim(0, (count+1))
        plt.xticks(np.arange(1.75, count, step=1.5), lsSubLvs1, rotation=0)
        plt.ylim((ylim_min-1), (ylim_max+1))
        plt.ylabel('Absolute %s Power (\u03BCV\u00b2)' %frequency)
        plt.title('Electrode %s' % eletrodo)
        plt.axhline(ylim_min - 1, color='k')
        plt.axvline(0, color='k')

        # Legend Box
        patches = []
        for (i, l) in zip(fill_list, lsSubLvs2):
            if (i == 'full'):
                fcolor = 'black'
            else:
                fcolor = 'white'
            patches = patches + [mpatches.Patch(edgecolor='black', facecolor=fcolor, label=l)]
        plt.legend(handles=patches) # for legend title: title='IndVar2'

        # Save plot to file
        your_vector_file_name = ('PlotInteraction_%s.eps' %eletrodo)
        your_raster_file_name = ('PlotInteraction_%s.png' %eletrodo)
        save_vector_file = os.path.join(plot_directory, your_vector_file_name)
        save_raster_file = os.path.join(plot_directory, your_raster_file_name)
        plt.savefig(save_vector_file)
        plt.savefig(save_raster_file, dpi=300)
        plt.close()
    #prints end feedback
    print("All Interaction Plots saved!")
    
    # IndVar1 Plots
    for eletrodo in lista_eletrodos:
        count = 0
        for sublevel in keys1:
            count += 1
            fmt_stl = []
            ls_stl = []
            for i in range(len(indVar1_SubLevels)):
              fmt_stl.append(icon_list[i])
              ls_stl.append(fill_list[1])
            x = sublevel[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            plt.errorbar(count, means, std, fmt=fmt_stl[count-1], fillstyle=ls_stl[count-1], lw=1, capsize=4, ecolor='k', markeredgewidth=1, mec='k')

        # Axes and Labels
        plt.box(on=False)
        plt.xlim(0, (count + 1))
        plt.xticks(range(1, (count + 1)), lsSubLvs1, rotation=0)
        plt.ylim((ylim_min - 1), (ylim_max + 1))
        plt.ylabel('Absolute %s Power (\u03BCV\u00b2)' %frequency)
        plt.title('Electrode %s' % eletrodo)
        plt.axhline(ylim_min - 1, color='k')
        plt.axvline(0, color='k')

        # Save plot to file
        your_vector_file_name = ('PlotIndVar1_%s.eps' %eletrodo)
        your_raster_file_name = ('PlotIndVar1_%s.png' %eletrodo)
        save_vector_file = os.path.join(plot_directory, your_vector_file_name)
        save_raster_file = os.path.join(plot_directory, your_raster_file_name)
        plt.savefig(save_vector_file)
        plt.savefig(save_raster_file, dpi=300)
        plt.close()
    #prints end feedback
    print("All IndVar1 Plots saved!")
    
    # IndVar2 Plots
    for eletrodo in lista_eletrodos:
        count = 0
        for sublevel in keys2:
            count += 1
            fmt_stl = []
            ls_stl = []
            for i in range(len(indVar2_SubLevels)):
              fmt_stl.append(icon_list[0])
              ls_stl.append(fill_list[i])
            x = sublevel[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            plt.errorbar(count, means, std, fmt=fmt_stl[count-1], fillstyle=ls_stl[count-1], lw=1, capsize=4, ecolor='k', markeredgewidth=1, mec='k')

        # Axes and Labels
        plt.box(on=False)
        plt.xlim(0, (count+1))
        plt.xticks(range(1, (count+1)), lsSubLvs2, rotation=0)
        plt.ylim((ylim_min-1), (ylim_max+1))
        plt.ylabel('Absolute %s Power (\u03BCV\u00b2)' % frequency)
        plt.title('Electrode %s' % eletrodo)
        plt.axhline(ylim_min - 1, color='k')
        plt.axvline(0, color='k')

        # Save plot to file
        your_vector_file_name = ('PlotIndVar2_%s.eps' %eletrodo)
        your_raster_file_name = ('PlotIndVar2_%s.png' %eletrodo)
        save_vector_file = os.path.join(plot_directory, your_vector_file_name)
        save_raster_file = os.path.join(plot_directory, your_raster_file_name)
        plt.savefig(save_vector_file)
        plt.savefig(save_raster_file, dpi=300)
        plt.close()
    #prints end feedback
    print("All IndVar2 Plots saved!")
    
    #Plot IndVar1 vs IndVar2
    for eletrodo in lista_eletrodos:
        count = 0
        x_v = {}
        y_v = {}
        max_mean = 0
        min_mean = means
        subLvsid1subLvsid2 = {}
        m = []
        xv = []
        yv = []

        # IndVar1IndVar2 separated by IndVar Subleves
        for id2sblvs in range(len(lsSubLvs2)): # loop para entrada no pre / pos
          count = 0
          exec('x%i_values = []'%id2sblvs)
          exec('y%i_values = []'%id2sblvs)
          for id1sblvs in range(len(lsSubLvs1)): # loop para cada tarefa
            count += 1
            exec('x%i_values=np.append(x%i_values, [count])'%(id2sblvs,id2sblvs))
            x_v['x%i'%id2sblvs] = ('x%i_values'%id2sblvs) 
            exec ('m.clear()')
            exec ('m.append((data.loc[indVar1 == %d]).loc[indVar2 == %d][eletrodo])'%(id1sblvs+1, id2sblvs+1))
            means = np.mean(m)
            exec('y%i_values=np.append(y%i_values, [means])'%(id2sblvs,id2sblvs))
            y_v['y%i'%id2sblvs] = ('y%i_values'%id2sblvs)
            if (means > max_mean):
              max_mean = means
            elif (means < min_mean):
              min_mean = means
        for value in list(x_v.values()):
          xv.append(eval(value))
        for value in list(y_v.values()):
          yv.append(eval(value))
        
        # Axes and Labels
        plt.xlim(0, (count+1))
        plt.xticks(range(1, (count+1)), lsSubLvs1, rotation=0)
        plt.ylim((ylim_min-1), (ylim_max+1))
        plt.ylabel('Estimated marginal means')
        plt.title('Electrode %s' %eletrodo)
        plt.axhline(ylim_min - 1, color='k')
        plt.axvline(0, color='k')
        for a, b in zip(range(len(xv)),range(len(yv))):
          plt.plot(xv[a], yv[b], c=color_list[a], label=(lsSubLvs2[b]))
        plt.legend()
        
        # Save plot to file
        your_vector_file_name = ('PlotIndVar1IndVar2_%s.eps' %eletrodo)
        your_raster_file_name = ('PlotIndVar1IndVar2_%s.png' %eletrodo)
        save_vector_file = os.path.join(plot_directory, your_vector_file_name)
        save_raster_file = os.path.join(plot_directory, your_raster_file_name)
        plt.savefig(save_vector_file)
        plt.savefig(save_raster_file, dpi=300)
        plt.close()

    #prints end feedback
    print("All IndVar1IndVar2 Plots saved!")
    print("All Plots saved!")
    

Define a função que gera os Box plots

In [0]:
def boxPlot(data):
    # Checks if results dir exists and creates it if necessary
    if not os.path.exists(path + '/results'):
        os.makedirs(path + '/results')
    if not os.path.exists(path + '/results/boxplots/'):
        os.makedirs(path + '/results/boxplots/')

    # Calc y axe limits
    ylim_min = 0
    ylim_max = 0
    for eletrodo in lista_eletrodos:
        for k in keys:
            minvalue = np.amin(k[eletrodo])
            maxvalue = np.amax(k[eletrodo])
            if (minvalue < ylim_min):
                ylim_min = minvalue
            elif (maxvalue > ylim_max):
                ylim_max = maxvalue

    # Export path
    plot_directory = path + '/results/boxplots/'
    totalLen = len(lsSubLvs)
    for eletrodo in lista_eletrodos:
        fig, ax1 = plt.subplots(figsize=(10, 6))
        ax1.boxplot([k[eletrodo] for k in keys], widths=0.1)

        # Begining Styling of Plot
        fig.canvas.set_window_title('Eletrodo: %s' %eletrodo)
        fig.subplots_adjust(left=0.075, right=0.95, top=0.9, bottom=0.25)

        # Add a horizontal grid to the plot, but make it very light in color
        # so we can use it for reading data values but not be distracting
        ax1.yaxis.grid(True, linestyle='-', which='major', color='lightgrey', alpha=0.5)

        # Hide these grid behind plot objects
        ax1.set_axisbelow(True)
        ax1.set_title('Electrode: %s' %eletrodo)
        ax1.set_xlabel('IndVar1 / IndVar2', labelpad=20)
        ax1.set_ylabel('Value', labelpad=10)

        # Set the axes ranges and axes labels
        ax1.set_xlim(0.5, totalLen + 0.5)
        ax1.set_ylim((ylim_min-1), (ylim_max+1))
        ax1.set_xticklabels(lsSubLvs, fontsize=8, horizontalalignment='center')

        plt.box(on=False)
        plt.axhline((ylim_min-1), color='k')
        plt.axvline(.5, color='k')

        # Save plot to file
        your_vector_file_name = ('BoxPlot_%s.eps' %eletrodo)
        your_raster_file_name = ('BoxPlot_%s.png' %eletrodo)
        save_vector_file = os.path.join(plot_directory, your_vector_file_name)
        save_raster_file = os.path.join(plot_directory, your_raster_file_name)
        plt.savefig(save_vector_file)
        plt.savefig(save_raster_file, dpi=300)
        plt.close()
    print ('Boxplots done!')

Inicio da execução das funções

Cria um arquivo com os resultados dos cálculos

In [0]:
'''Creates the final report'''
outputfile = open((path + '/results/final_report.txt'), 'w')

Remove os outliers

In [0]:
remove_outliers(data)

Atualiza a base de dados para a planilha sem outliers

In [0]:
data = pd.read_csv(path + '/results/final_sheet_wo_outliers.csv', index_col=False) # Updates data to final sheet without outliers

Baseado na quantidade de subníveis da variável independente 2, decide entre Two-way ANOVA com Teste-T e Two-way ANOVA com One-way ANOVA.

In [0]:
# Based on the lengh of IndVar2 decide to run T-test or One-way ANOVA

if len(lsSubLvs2) < 3:
  run_stats_twa_tt(data) # runs two-way anova with t-test
  print('two-way anova with t-test choosen')
else:
  run_stats_twa_owa(data) # runs two-way anova with one-way anova
  print('two-way anova with one-way anova choosen')

Cria e salva os Box plot

In [0]:
boxPlot(data)

Cria e salva os demais Plots

In [0]:
plot_to_files(data)

Encerra o arquivo com o resultado dos calculos estatísticos 

In [0]:
outputfile.close()

Cria um relatório interativo da estatística da base de dados utilizando o Pandas Profiling 

In [0]:
profile = ProfileReport(data, title='Relatório - Pandas Profiling', html={'style':{'full_width':True}})
profile.to_file(output_file=(path + "/results/PandasProfiling.html"))

# ---

In [0]:
print('ENDED SUCCESSFULLY') # prints feedback of the end of the script

**LEMBRE-SE**

Verifique a pasta 'results' que foi criada no mesmo local da pasta 'your_folders_here' para analisar seus resultados.

Obrigado por utilizar o script e não se esqueça de nos mencionar em seu artigo.